In [23]:
import numpy as np
import pickle
import pandas as pd

In [24]:
with open("../model/word2vec_model_avi.pkl", "rb") as f:
    word2vec_model = pickle.load(f)

In [25]:
query_sentence = 'a crime story with a beautiful woman' 

pd.options.display.max_colwidth = 500

In [28]:
# reading in dataset
df = pd.read_csv("../data/tok_lem_sentence.csv")
df.head()

,original_title,genres,sentence,tok_lem_sentence
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]","Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.","['led', 'by', 'woody', 'andy', 'toy', 'live', 'happily', 'in', 'his', 'room', 'until', 'andy', 'birthday', 'brings', 'buzz', 'lightyear', 'onto', 'the', 'scene', 'afraid', 'of', 'losing', 'his', 'place', 'in', 'andy', 'heart', 'woody', 'plot', 'against', 'buzz', 'but', 'when', 'circumstance', 'separate', 'buzz', 'and', 'woody', 'from', 'their', 'owner', 'the', 'duo', 'eventually', 'learns', 'to', 'put', 'aside', 'their', 'difference']"
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]","When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.","['when', 'sibling', 'judy', 'and', 'peter', 'discover', 'an', 'enchanted', 'board', 'game', 'that', 'open', 'the', 'door', 'to', 'a', 'magical', 'world', 'they', 'unwittingly', 'invite', 'alan', 'an', 'adult', 'who', 'been', 'trapped', 'inside', 'the', 'game', 'for', '26', 'year', 'into', 'their', 'living', 'room', 'alan', 'only', 'hope', 'for', 'freedom', 'is', 'to', 'finish', 'the', 'game', 'which', 'prof', 'risky', 'a', 'all', 'three', 'find', 'themselves', 'running', 'from', 'giant', 'rh..."
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]","A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.","['a', 'family', 'wedding', 'reignites', 'the', 'ancient', 'feud', 'between', 'next', 'door', 'neighbor', 'and', 'fishing', 'buddy', 'john', 'and', 'max', 'meanwhile', 'a', 'sultry', 'italian', 'divorcée', 'open', 'a', 'restaurant', 'at', 'the', 'local', 'bait', 'shop', 'alarming', 'the', 'local', 'who', 'worry', 'she', 'll', 'scare', 'the', 'fish', 'away', 'but', 'she', 'le', 'interested', 'in', 'seafood', 'than', 'she', 'is', 'in', 'cooking', 'up', 'a', 'hot', 'time', 'with', 'max']"
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]","Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.","['cheated', 'on', 'mistreated', 'and', 'stepped', 'on', 'the', 'woman', 'are', 'holding', 'their', 'breath', 'waiting', 'for', 'the', 'elusive', 'good', 'man', 'to', 'break', 'a', 'string', 'of', 'le', 'than', 'stellar', 'lover', 'friend', 'and', 'confidant', 'vannah', 'bernie', 'glo', 'and', 'robin', 'talk', 'it', 'all', 'out', 'determined', 'to', 'find', 'a', 'better', 'way', 'to', 'breathe']"
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.","

In [21]:
def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

In [29]:
def is_word_in_model(word, model):
    """
    Check on individual words ``word`` that it exists in ``model``.
    """
    assert type(model).__name__ == 'KeyedVectors'
    is_in_vocab = word in model.key_to_index.keys()
    return is_in_vocab

def predict_w2v(query_sentence, dataset, model, topk=3):
    query_sentence = query_sentence.split()
    in_vocab_list, best_index = [], [0]*topk
    for w in query_sentence:
        # remove unseen words from query sentence
        if is_word_in_model(w, model.wv):
            in_vocab_list.append(w)
    # Retrieve the similarity between two words as a distance
    if len(in_vocab_list) > 0:
        sim_mat = np.zeros(len(dataset))  # TO DO
        for i, data_sentence in enumerate(dataset):
            if data_sentence:
                sim_sentence = model.wv.n_similarity(
                        in_vocab_list, data_sentence)
            else:
                sim_sentence = 0
            sim_mat[i] = np.array(sim_sentence)
        # Take the five highest norm
        best_index = np.argsort(sim_mat)[::-1][:topk]
    return best_index

# Predict
best_index = predict_w2v(query_sentence, df['tok_lem_sentence'].values, word2vec_model)    
display(df[['original_title', 'genres', 'sentence']].iloc[best_index])

,original_title,genres,sentence
1473,The Next Step,[],Backstage at a Broadway show.
17848,Pilatus und andere - Ein Film für Karfreitag,[],TV Drama from Wajda
13310,Game Over,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]",...
